In [333]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [334]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [335]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [336]:
df_train = pd.read_csv('../input/aviakompaniya/train_dataset.csv')
df_test = pd.read_csv('../input/aviakompaniya/test_dataset.csv')

In [337]:
df_train.shape

In [338]:
df_test.shape

In [339]:
df_train.info()

In [340]:
df_test.info()

In [341]:
df_train[df_train['Arrival Delay in Minutes'].isna()]

In [342]:
df_train['Arrival Delay in Minutes'].value_counts()

In [343]:
df_train.sample(10)

In [344]:
df_train['Arrival Delay in Minutes'].fillna(df_train['Departure Delay in Minutes'],inplace=True)
df_test['Arrival Delay in Minutes'].fillna(df_test['Departure Delay in Minutes'],inplace=True)

In [345]:
df_train[['Gender','Customer Type','Type of Travel','Class']].nunique()

In [346]:
def str_int(df):
    gender_mapping = {
           'Male': 1,
           'Female':0}
    df['Gender'] = df['Gender'].map(gender_mapping)
    customer_mapping = {
           'Loyal Customer': 1,
           'disloyal Customer':0}
    df['Customer Type'] = df['Customer Type'].map(customer_mapping)
    class_mapping = {
           'Eco Plus': 1,
           'Eco':0,
           'Business':2}
    df['Class'] = df['Class'].map(class_mapping)
    travel_mapping = {
           'Personal Travel': 0,
           'Business travel':1}
    df['Type of Travel'] = df['Type of Travel'].map(travel_mapping)
    return df

In [347]:
df_train = str_int(df_train)
df_test = str_int(df_test)

In [348]:

facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Age',shade= True)
facet.set(xlim=(0, df_train['Age'].max()))
facet.add_legend()
plt.grid()
plt.show()

In [349]:
facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Flight Distance',shade= True)
facet.set(xlim=(0, df_train['Flight Distance'].max()))
facet.add_legend()
plt.grid()
plt.show()

In [350]:
facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Arrival Delay in Minutes',shade= True)
facet.set(xlim=(0, df_train['Arrival Delay in Minutes'].max()))
facet.add_legend()
plt.grid()
plt.show()

In [351]:
facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Arrival Delay in Minutes',shade= True)
facet.set(xlim=(0, df_train['Arrival Delay in Minutes'].max()))
facet.add_legend()
plt.xlim(0, 100)
plt.grid()
plt.show()

In [352]:
facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Departure Delay in Minutes',shade= True)
facet.set(xlim=(0, df_train['Departure Delay in Minutes'].max()))
facet.add_legend()
plt.grid()
plt.show()

In [353]:
facet = sns.FacetGrid(data=df_train, hue="satisfaction",aspect=4)
facet.map(sns.kdeplot,'Departure Delay in Minutes',shade= True)
facet.set(xlim=(0, df_train['Departure Delay in Minutes'].max()))
facet.add_legend()
plt.xlim(0,100)
plt.grid()
plt.show()

In [354]:
def group_v(df):
    df.loc[ df['Age'] <= 18, 'Age'] = 0
    df.loc[(df['Age'] > 18) & (df['Age'] <= 36), 'Age'] = 1
    df.loc[(df['Age'] > 36) & (df['Age'] <= 62), 'Age'] = 2
#     df.loc[(df['Age'] > ) & (df['Age'] <= 50), 'Age'] = 3
#     df.loc[(df['Age'] > 50) & (df['Age'] <= 65), 'Age'] = 2
    df.loc[ df['Age'] > 62, 'Age'] = 0
    df.loc[ df['Flight Distance'] <= 1400, 'Flight Distance'] = 0
    df.loc[(df['Flight Distance'] > 1400) & (df['Flight Distance'] <= 2000), 'Flight Distance'] = 2
    df.loc[(df['Flight Distance'] > 2000) & (df['Flight Distance'] <= 4000), 'Flight Distance'] = 2
    df.loc[ df['Flight Distance'] > 4000, 'Flight Distance'] = 1
#     df.loc[ df['Flight Distance'] <= 300, 'Flight Distance'] = 0
#     df.loc[(df['Flight Distance'] > 300) & (df['Flight Distance'] <= 700), 'Flight Distance'] = 1
#     df.loc[(df['Flight Distance'] > 700) & (df['Flight Distance'] <= 1250), 'Flight Distance'] = 0
#     df.loc[(df['Flight Distance'] > 1250) & (df['Flight Distance'] <= 1550), 'Flight Distance'] = 1
#     df.loc[(df['Flight Distance'] > 1550) & (df['Flight Distance'] <= 2500), 'Flight Distance'] = 2
#     df.loc[ df['Flight Distance'] > 2500, 'Flight Distance'] = 1
    df.loc[ df['Arrival Delay in Minutes'] <= 5, 'Arrival Delay in Minutes'] = 0
    df.loc[(df['Arrival Delay in Minutes'] > 5) & (df['Arrival Delay in Minutes'] <= 15), 'Arrival Delay in Minutes'] = 1
    df.loc[(df['Arrival Delay in Minutes'] > 15) & (df['Arrival Delay in Minutes'] <= 25), 'Arrival Delay in Minutes'] = 2
    df.loc[ df['Arrival Delay in Minutes'] > 25, 'Arrival Delay in Minutes'] = 3
    df.loc[ df['Departure Delay in Minutes'] <= 5, 'Departure Delay in Minutes'] = 0
    df.loc[(df['Departure Delay in Minutes'] > 5) & (df['Departure Delay in Minutes'] <= 15), 'Departure Delay in Minutes'] = 1
    df.loc[(df['Departure Delay in Minutes'] > 15) & (df['Arrival Delay in Minutes'] <= 25), 'Departure Delay in Minutes'] = 2
    df.loc[ df['Departure Delay in Minutes'] > 25, 'Departure Delay in Minutes'] = 3
    return df

In [355]:
df_train1 = group_v( df_train )
df_test= group_v( df_test )

In [356]:
df_train1.describe().T

In [357]:
df_train1.corrwith(df_train1['satisfaction'])

In [358]:
corr_matrix = df_train.corr().abs()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [359]:
train = df_train.drop(['id','Gender','Gate location','Departure Delay in Minutes','Departure/Arrival time convenient'],axis=1)
test = df_test.drop(['id','Gender','Gate location','Departure Delay in Minutes','Departure/Arrival time convenient'],axis=1)

In [360]:
X_to_model = train.drop('satisfaction', axis=1).values
Y_to_model = train['satisfaction']
X_test = test.values

In [361]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [362]:
# clf = KNeighborsClassifier()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [363]:
# round(np.mean(score)*100, 2)

In [364]:
# clf = DecisionTreeClassifier()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [365]:
round(np.mean(score)*100, 2)


In [366]:
clf = RandomForestClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [367]:
round(np.mean(score)*100, 2)


In [368]:
# clf = GaussianNB()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [369]:
# round(np.mean(score)*100, 2)

In [370]:
# clf = SVC()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [371]:
# round(np.mean(score)*100, 2)


In [372]:
# clf = AdaBoostClassifier()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [373]:
# round(np.mean(score)*100, 2)

In [374]:
# clf = MLPClassifier()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [375]:
# round(np.mean(score)*100, 2)

In [383]:
clf = GaussianProcessClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [384]:
round(np.mean(score)*100, 2)

In [378]:
clf = RBF()
scoring = 'accuracy'
score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

In [379]:
# clf = QuadraticDiscriminantAnalysis()
# scoring = 'accuracy'
# score = cross_val_score(clf, X_to_model, Y_to_model, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)

In [380]:
# round(np.mean(score)*100, 2)

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_to_model, Y_to_model)
y_predict = rf_model.predict(X_test)

In [ ]:
submission = pd.DataFrame(y_predict,columns=['satisfaction'])
submission['id'] = df_test['id']
submission = submission[['id','satisfaction']]

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
submission